In [1]:
import os
import os.path as osp
import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import pickle as pkl

In [2]:
meta_root = '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/frame_records_csvs'
verify_video_root = '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/verify_videos'
out_video_root = '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/verify_videos_split'

driver_ver_dct ={
    'atalie': 1,
    'keli': 1,
    'matt': 1,
    'subject5_cap': 2,
    'subject5_tied_hair': 2,
    'subject25_cap': 2,
    'subject25_tied_hair': 2,
}

def get_image_list(dn, ver):
    if ver == 1:
        image_list_record_root = '/prj/qct/mmrd-cv/wonderland_data/3DFR_Data_wrap4d/Pair_man/PAC/meta/train'
    elif ver == 2:
        image_list_record_root = '/mnt/QPFA-LV/dataset/LightCage_Process_MV_Data/PAC_Add_Nose/meta'

    meta_file = osp.join(image_list_record_root, f'data_parsed_List_{dn}_man_process_list_train.csv')
    if ver == 1:
        image_filename_list = pd.read_csv(meta_file)['tar_bg'].tolist()
    elif ver == 2:
        image_filename_list = pd.read_csv(meta_file)['tar'].tolist()
    return image_filename_list

def get_motion_name(path):
    motion_name = path.split('/')[1]
    return motion_name

In [5]:
# dn, ver = list(driver_ver_dct.items())[0]

for  dn, ver in list(driver_ver_dct.items())[1:]:
    image_filename_list = get_image_list(dn, ver)
    merged_video_root = osp.join(verify_video_root, f'{dn}-driving')
    merged_video_list = glob.glob(osp.join(merged_video_root, '*.mp4'))
    source_names = [osp.basename(x).split('--')[0] for x in merged_video_list]
    for sn, merged_video_path in zip(source_names, merged_video_list):
        # open the merged_video using cv2
        cap = cv2.VideoCapture(merged_video_path)
        # get the frame rate of the video
        fps = cap.get(cv2.CAP_PROP_FPS)
        # get the total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # get the width and height of the video
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        motion = get_motion_name(image_filename_list[0])
        out_video_path = osp.join(out_video_root, dn, sn, f'{motion}.mp4')
        os.makedirs(osp.dirname(out_video_path), exist_ok=True)
        # create a VideoWriter object to save the output video
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(out_video_path, fourcc, fps, (width, height))

        # loop through the frames of the video
        for i in range(total_frames):
            motion_new = get_motion_name(image_filename_list[i])
            if motion_new != motion:
                out.release()
                motion = motion_new
                out_video_path = osp.join(out_video_root, dn, sn, f'{motion}.mp4')
                os.makedirs(osp.dirname(out_video_path), exist_ok=True)
                out = cv2.VideoWriter(out_video_path, fourcc, fps, (width, height))

            # read the next frame
            ret, frame = cap.read()
            # check if the frame was read successfully
            if not ret:
                break
            # write the frame to the output video
            out.write(frame)
        # release the VideoCapture and VideoWriter objects
        cap.release()
        out.release()